In [35]:
import gradio as gr
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv(override=True)

client = OpenAI()

gemini_client = OpenAI(
    base_url= "https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=os.getenv("GOOGLE_API_KEY")
)



In [4]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"

In [5]:
def stream_gpt(prompt):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response


In [36]:
def stream_gemini(prompt):
    stream = gemini_client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response


In [8]:
stream_gpt("time now")

<generator object stream_gpt at 0x111aeb840>

In [25]:
input_text = gr.Textbox(label="Enter a prompt",lines=10)
output_text = gr.Markdown(label="Response")
model_switch =gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")



In [31]:
def update_model(prompt,model):
    if model == "GPT":
        response = stream_gpt(prompt)
    elif model == "Gemini":
        response = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from response
    
    

In [37]:
view = gr.Interface(fn=update_model, inputs=[input_text,model_switch], outputs=output_text, title="LLM Playground", description="Enter a prompt and select a model to see the response",
flagging_mode="never",
examples=[
            ["Explain the Transformer architecture to a layperson", "GPT"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Gemini"]
        ], 
)
view.launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


In [49]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from scrapper import fetch_website_links, fetch_website_contents
from IPython.display import Markdown, display ,update_display
import json

In [50]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [51]:
def get_link_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links=fetch_website_links("https://www.apple.com")
    user_prompt += "\n".join(links)
    return user_prompt

In [52]:
def generate_usefull_links(url):
    user_prompt = get_link_user_prompt(url)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)


In [53]:
def generate_page_data_with_links(url):
    links = generate_usefull_links(url)
    page_data = fetch_website_contents(url)
    result= f"## landing page\n\n{page_data}\n\n## links\n"
    for link in links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link['url'])
    return result

In [62]:
def brochure_system_prompt(type) :
  return f"You are an assistant that analyzes the contents of several relevant pages from a company website\
and creates a short neat {type} brochure about the company for prospective customers, investors and recruits.\
Respond in markdown without code blocks.\
Include details of company culture, customers and careers/jobs if you have the information.\
"

In [63]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += generate_page_data_with_links(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt


## Frontend for brochure APP

In [64]:
def stream_brochure(company_name, url,type):
    stream = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt(type)},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [67]:
company_name = gr.Textbox(label="Company Name", lines=1)
url = gr.Textbox(label="Company URL", lines=1)
type= gr.Dropdown(["Serious", "Creative","Casual","Technical","Funny"], label="Select type", value="Serious")
response = gr.Markdown(label="Brochure")



In [66]:
view = gr.Interface(fn=stream_brochure, inputs=[company_name, url,type], outputs=response, title="Brochure Generator", description="Enter a company name and URL to generate a brochure"
,flagging_mode="never",examples=[["Apple", "https://www.apple.com","Serious"]])
view.launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.
